In [1]:
import time
from functools import wraps


def timeit(fn):
    """计算函数运行时间的装饰器"""

    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print(f"@timeit: {fn.__name__} took {t2 - t1: .5f} s")
        return result

    return measure_time

In [2]:
# 下面两行用不到可以注掉，因为我用的是VS Code，不知道怎么回事找不到模块要自己添加一下
# import sys
# sys.path.append('/Users/zhouxinyu/Code/Coding/zf_back_py')

import cv2
import numpy as np

class TubeBendingDetect:

    # 裁剪完成后进行分割
    def image_cut(self, img):
        # 从上到下，从左到右，不写就是原始的上下左右边界。带 - 号的是从右边或者从下开始计算距离，-10指切去右边缘长度为10的margin
        img = img[:, :-10, :]
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        lower_blue = np.array([100, 43, 46])
        upper_blue = np.array([124, 255, 255])
        # mask：两个阈值内的变白，其他的变黑，返回一个二值化的图像
        mask = cv2.inRange(hsv, lower_blue, upper_blue)
        #     plt_show(mask)
        img_morph = mask.copy()
        # 腐蚀膨胀，中值滤波
        core = (3, 3)
        cv2.erode(img_morph, core, img_morph, iterations=4)
        #     plt_show(img_morph)
        cv2.dilate(img_morph, core, img_morph, iterations=1)
        #     plt_show(img_morph)
        img_morph = cv2.medianBlur(img_morph, 15)
        #     plt_show(img_morph)
        res = cv2.findContours(img_morph, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        cnts = res[0]  # cnts就是提取到的所有contours组成的list

        # sort the elements in contours by the area of a contour, descending order
        cnts_sort = sorted(cnts, key=cv2.contourArea, reverse=True)
        # 生成一个与img相同shape的全0数组white_img
        white_img = np.zeros_like(img)
        #     plt_show0(white_img)
        white_img[:, :, :] = 0  # 变成一张纯黑的图
        # 第三个参数0代表：绘制cnts_sort里的第一个contour
        # 最后一个参数-1代表：绘制contour时，填充轮廓内部
        cv2.drawContours(white_img, cnts_sort, 0, (255, 255, 255), -1)  # 会在白底上画出黑色的区域
        #     plt_show0(white_img)
        black_img = np.zeros_like(img)
        black_img[:, :, :] = 255  # 变成一张纯白的图
        #     plt_show0(black_img)
        black_img_copy = cv2.cvtColor(black_img, cv2.COLOR_BGR2GRAY)  # 变成了一张纯黑的图
        #     plt_show(black_img_copy)
        cv2.drawContours(black_img_copy, cnts_sort, 0, (0, 0, 0),
                         -1)  # 这里的颜色只要不是(255, 255, 255)，就都是白色；(255, 255, 255)画出黑色
        #     cv2.imwrite('black_img.png', black_img_copy)
        #     plt_show(black_img_copy)
        res = cv2.findContours(black_img_copy, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

        #     cv2.imwrite('black_img.png', black_img_copy)
        img_copy = img.copy()
        cv2.drawContours(img_copy, res[0][0], -1, (0, 255, 0), 10)
        #     cv2.imwrite('img_copy.png', img_copy)
        #     plt_show0(img_copy)
        x, y, w, h = cv2.boundingRect(res[0][0])
        pure_img = img.copy()
        # 获得方框
        cv2.rectangle(pure_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #     cv2.imwrite('pure_img.png', pure_img)
        #     plt_show0(pure_img)
        ## 灰度图
        gray_pure_img = cv2.cvtColor(pure_img, cv2.COLOR_BGR2GRAY)
        #     cv2.imwrite('gray_pure_img.png', gray_pure_img)
        #     plt_show(gray_pure_img)
        # 裁剪
        cuted_image = img[y:y + h, x:x + w]
        #     cv2.imwrite('cuted_image.png', cuted_image)
        #     plt_show0(cuted_image)
        bounding = res[0][0]  # 最大的contour

        # 让contour的所有点的坐标移动，以配合裁剪出来的新图片
        def bounding_move(bounding_point):
            bounding_point[0, 0] -= x  # 裁剪范围的左上角的x坐标
            bounding_point[0, 1] -= y
            return bounding_point

        bounding = np.array(list(map(bounding_move, bounding)))
        return cuted_image, bounding

    def fill_black(self, image, bounding):
        stencil = np.zeros(image.shape).astype(image.dtype)
        color = (255, 255, 255)
        cv2.fillPoly(stencil, [bounding], color)
        result = cv2.bitwise_and(image, stencil)
        return result

    def delete_white(self, rawImage):
        #     plt_show0(rawImage)
        # 高斯去噪
        image = cv2.GaussianBlur(rawImage, (3, 3), 0)
        # plt_show0(image)

        height, width, color = image.shape

        image_hsv = image.copy()
        image_hsv = cv2.cvtColor(image_hsv, cv2.COLOR_RGB2HSV)

        lower_blue = np.array([0, 0, 180])
        upper_blue = np.array([180, 255, 255])
        # mask：两个阈值内的变白，其他的变黑，返回一个二值化的图像
        mask = cv2.inRange(image_hsv, lower_blue, upper_blue)
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

        # plt_show0(mask)

        stencil = cv2.bitwise_not(mask)
        # plt_show0(stencil)

        image = cv2.bitwise_and(image, stencil)
        # plt_show0(image)

        xPosition = 0
        for x in range(0, width, 5):
            cnt = 0
            for y in range(0, height):
                if image[y][x][0] > 0:
                    cnt += 1
            if cnt > 125:
                xPosition = x
                break
        #     print("xPos",xPosition)
        return xPosition

    def get_two_lines(self, fill_black_right):
        height, width, color = fill_black_right.shape
        fill_black_right_HSV = cv2.cvtColor(fill_black_right, cv2.COLOR_BGR2HSV)

        # left 1 ---------------------------------------
        left_1_middle_y_1 = 0
        left_1_middle_y_2 = 0
        left_1_middle_y = 0
        left_1_start_x = width // 20
        left_1_end_x = width // 10
        cnt = 0
        cnt_2 = 0
        for x in range(left_1_start_x, left_1_end_x, 5):
            for y in range(0, height):
                if fill_black_right_HSV[y][x][2] != 0:
                    cnt += 1
                    left_1_middle_y_1 += y
                    break
            for k in range(height - 1, 0, -1):
                if fill_black_right_HSV[k][x][2] != 0:
                    cnt_2 += 1
                    left_1_middle_y_2 += k
                    break
        left_1_middle_y_1 = left_1_middle_y_1 // cnt
        left_1_middle_y_2 = left_1_middle_y_2 // cnt_2

        # print(left_1)
        left_1_middle_x = (left_1_start_x + left_1_end_x) // 2
        left_1_middle_y = (left_1_middle_y_1 + left_1_middle_y_2) // 2
        #     print(left_1_middle_x,left_1_middle_y)

        # left 2 ------------------------------------------
        left_2_middle_y = 0
        left_2_middle_y_1 = 0
        left_2_middle_y_2 = 0
        left_2_start_x = 3 * width // 20
        left_2_end_x = width // 5
        cnt = 0
        cnt_2 = 0
        for x in range(left_2_start_x, left_2_end_x, 5):
            for y in range(0, height):
                if fill_black_right_HSV[y][x][2] != 0:
                    cnt += 1
                    left_2_middle_y_1 += y
                    break
            for k in range(height - 1, 0, -1):
                if fill_black_right_HSV[k][x][2] != 0:
                    cnt_2 += 1
                    left_2_middle_y_2 += k
                    break
        left_2_middle_y_1 = left_2_middle_y_1 // cnt
        left_2_middle_y_2 = left_2_middle_y_2 // cnt_2

        # print(left_1)
        left_2_middle_x = (left_2_start_x + left_2_end_x) // 2
        left_2_middle_y = (left_2_middle_y_1 + left_2_middle_y_2) // 2
        #     print(left_2_middle_x,left_2_middle_y)

        # right 1 ------------------------------------------
        right_1 = []
        right_1_middle_y = 0
        right_1_middle_y_1 = 0
        right_1_middle_y_2 = 0
        right_1_start_x = width - width // 5
        right_1_end_x = width - 3 * width // 20
        cnt = 0
        cnt_2 = 0
        for x in range(right_1_start_x, right_1_end_x, 5):
            for y in range(0, height):
                if fill_black_right_HSV[y][x][2] != 0:
                    cnt += 1
                    right_1_middle_y_1 += y
                    break
            for k in range(height - 1, 0, -1):
                if fill_black_right_HSV[k][x][2] != 0:
                    cnt_2 += 1
                    right_1_middle_y_2 += k
                    break
        right_1_middle_y_1 = right_1_middle_y_1 // cnt
        right_1_middle_y_2 = right_1_middle_y_2 // cnt_2

        right_1_middle_x = (right_1_start_x + right_1_end_x) // 2
        right_1_middle_y = (right_1_middle_y_1 + right_1_middle_y_2) // 2
        #     print(right_1_middle_x,right_1_middle_y)

        # right 2 ------------------------------------------
        right_2 = []
        right_2_middle_y = 0
        right_2_middle_y_1 = 0
        right_2_middle_y_2 = 0
        right_2_start_x = width - width // 10
        right_2_end_x = width - width // 20
        cnt = 0
        cnt_2 = 0
        for x in range(right_2_start_x, right_2_end_x, 5):
            for y in range(0, height):
                if fill_black_right_HSV[y][x][2] != 0:
                    cnt += 1
                    right_2_middle_y_1 += y
                    break
            for k in range(height - 1, 0, -1):
                if fill_black_right_HSV[k][x][2] != 0:
                    cnt_2 += 1
                    right_2_middle_y_2 += k
                    break
        right_2_middle_y_1 = right_2_middle_y_1 // cnt
        right_2_middle_y_2 = right_2_middle_y_2 // cnt_2

        right_2_middle_x = (right_2_start_x + right_2_end_x) // 2
        right_2_middle_y = (right_2_middle_y_1 + right_2_middle_y_2) // 2
        #     print(right_2_middle_x,right_2_middle_y)
        return [[left_1_middle_x, left_1_middle_y], [left_2_middle_x, left_2_middle_y],
                [right_1_middle_x, right_1_middle_y], [right_2_middle_x, right_2_middle_y]]

    def vector_angle(self, line1: list, line2: list):
        """
        计算两条直线的夹角，
        :param line1: 直线一[x1, y1, x2, y2]
        :param line2: 直线二 [x1, y1, x2, y2]
        :return:
        """

        vector1 = np.array([line1[2] - line1[0], line1[3] - line1[1]])  # (x2-x1,y2-y1)
        vector2 = np.array([line2[2] - line2[0], line2[3] - line2[1]])
        return np.arccos(
            vector1 @ vector2 / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))

    @timeit
    def process_image(self, image):
        cuted_image, bounding = self.image_cut(image)
        fill_b = self.fill_black(cuted_image, bounding)
        xPosition = self.delete_white(fill_b)
        margin = 50
        height, width, color = fill_b.shape
        fill_black_right = fill_b[0:height, xPosition + margin:width - margin]
        # plt_show0(fill_black_right)
        fourpoints = self.get_two_lines(fill_black_right)
        line1 = [fourpoints[0][0], fourpoints[0][1], fourpoints[1][0], fourpoints[1][1]]
        line2 = [fourpoints[2][0], fourpoints[2][1], fourpoints[3][0], fourpoints[3][1]]
        a = self.vector_angle(line1, line2)
        # print(a)
        pi = 3.1415926
        result = a / pi * 180
        return bool(result > 5), result

tb = TubeBendingDetect()

In [3]:
image = cv2.imread(
            '/Users/zhouxinyu/Code/Python/jupyter/Project/ZF/数据测试/压线筒弯曲/数据/线筒弯曲 (1).tif')
is_right, degree = tb.process_image(image)
print("是否弯曲: ", is_right, degree)

@timeit: process_image took  0.88541 s
是否弯曲:  True 45.81845624326721


In [5]:
for i in range(1, 14):
    print(i)
    fileName = '/Users/zhouxinyu/Code/Python/jupyter/Project/ZF/数据测试/压线筒弯曲/数据/线筒弯曲 ('+str(i)+').tif'
    image = cv2.imread(fileName)
    is_right, degree = tb.process_image(image)
    print("是否弯曲: ", is_right, degree)
    # showImages([right_img])

1
@timeit: process_image took  0.86079 s
是否弯曲:  True 45.81845624326721
2
@timeit: process_image took  0.38159 s
是否弯曲:  True 42.87530049255115
3
@timeit: process_image took  0.43617 s
是否弯曲:  True 36.06548619520893
4
@timeit: process_image took  0.41578 s
是否弯曲:  True 39.1700507056491
5
@timeit: process_image took  0.47020 s
是否弯曲:  True 35.55112495519627
6
@timeit: process_image took  0.46698 s
是否弯曲:  True 27.88298407432556
7
@timeit: process_image took  0.69873 s
是否弯曲:  True 36.099383740330026
8
@timeit: process_image took  0.54455 s
是否弯曲:  True 36.1023731085026
9
@timeit: process_image took  0.39663 s
是否弯曲:  True 27.151018893941973
10
@timeit: process_image took  0.32084 s
是否弯曲:  True 24.180539063955344
11
@timeit: process_image took  0.31854 s
是否弯曲:  False 1.0406425927988072
12
@timeit: process_image took  0.28776 s
是否弯曲:  True 29.931322571514844
13
@timeit: process_image took  0.31667 s
是否弯曲:  True 26.824195111968127


In [4]:
for i in range(0, 21):
    print(i)
    fileName = '/Users/zhouxinyu/Code/Python/jupyter/Project/ZF/数据测试/压线筒弯曲/数据2/'+str(i)+'.jpg'
    image = cv2.imread(fileName)
    is_right, degree = tb.process_image(image)
    print("是否弯曲: ", is_right, degree)
    # showImages([right_img])

0
@timeit: process_image took  0.40862 s
是否弯曲:  True 10.770968120616267
1
@timeit: process_image took  0.27243 s
是否弯曲:  True 11.754025244128979
2
@timeit: process_image took  0.22696 s
是否弯曲:  True 11.144395798030448
3
@timeit: process_image took  0.22176 s
是否弯曲:  True 18.910173235337297
4
@timeit: process_image took  0.25349 s
是否弯曲:  True 18.923035740387494
5
@timeit: process_image took  0.27382 s
是否弯曲:  True 5.04331665689642
6
@timeit: process_image took  0.27912 s
是否弯曲:  False 1.4340363725424907
7
@timeit: process_image took  0.25905 s
是否弯曲:  True 6.768279620793613
8
@timeit: process_image took  0.25833 s
是否弯曲:  True 7.968692932903212
9
@timeit: process_image took  0.21847 s
是否弯曲:  True 7.9335157101279306
10
@timeit: process_image took  0.25113 s
是否弯曲:  True 9.76582090144086
11
@timeit: process_image took  0.22300 s
是否弯曲:  True 13.827135947861386
12
@timeit: process_image took  0.26783 s
是否弯曲:  True 14.109688026037416
13
@timeit: process_image took  0.27516 s
是否弯曲:  True 15.897490013